# REQUEST FILE GENERATION

In [1]:
import random
import networkx as nx
import pickle

def generate_requests(graph, num_requests, demand_type):
    result = []
    node_request = len(graph.nodes) 
    
    for _ in range(num_requests):
        source = random.choice(list(graph.nodes))
        destination = random.choice(list(graph.nodes))
        while destination == source:
            destination = random.choice(list(graph.nodes))
        if demand_type == 'variable':
            demand = 12.5 * random.choice([1, 2, 3, 4, 5, 6])
        else:
            demand = demand
        
        new_req = {
            "source": source,
            "destination": destination,
            "demand": demand
        }
        
        result.append(new_req)
        
    return result

if __name__ == "__main__":
    file_path = input("Enter file name : ")
    
    try:
        # graph = nx.read_gpickle(file_path+".gpickle")
        with open(file_path + ".gpickle", "rb") as f:
            graph = pickle.load(f)
        num_requests = int(input("Enter the number of requests: "))
        demand_type = input("Enter demand type (40 or 80 for fixed, or variable): ").lower()
        if demand_type == '40':
            demand = 40
            
            
        elif demand_type == '80':
            demand = 80
        elif demand_type == 'variable':
            demand = "variable"
        else:
            print("Invalid demand type.")
            exit()
        requests = generate_requests(graph, num_requests, demand_type)
        for req in requests:
            print(req)
    
    except FileNotFoundError:
        print("The specified gpickle file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


{'source': 6, 'destination': 2, 'demand': 62.5}
{'source': 10, 'destination': 20, 'demand': 50.0}
{'source': 22, 'destination': 3, 'demand': 12.5}
{'source': 2, 'destination': 3, 'demand': 37.5}
{'source': 12, 'destination': 15, 'demand': 12.5}
{'source': 4, 'destination': 23, 'demand': 12.5}
{'source': 4, 'destination': 6, 'demand': 25.0}
{'source': 14, 'destination': 16, 'demand': 75.0}
{'source': 23, 'destination': 10, 'demand': 25.0}
{'source': 17, 'destination': 12, 'demand': 37.5}
{'source': 20, 'destination': 0, 'demand': 50.0}
{'source': 7, 'destination': 5, 'demand': 37.5}
{'source': 11, 'destination': 5, 'demand': 25.0}
{'source': 8, 'destination': 10, 'demand': 25.0}
{'source': 6, 'destination': 0, 'demand': 25.0}
{'source': 14, 'destination': 9, 'demand': 12.5}
{'source': 3, 'destination': 14, 'demand': 37.5}
{'source': 11, 'destination': 13, 'demand': 25.0}
{'source': 9, 'destination': 13, 'demand': 37.5}
{'source': 6, 'destination': 7, 'demand': 75.0}
{'source': 21, 'dest

# DATA GENERATION

In [2]:
import networkx as nx

def load_network_graph(filename):
    return nx.read_gpickle(filename)

def sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity ):
    results = [] 
    total_regen_primary=0
    total_regen_backup=0
    Total_Frequency_Slot=0
    Guard_Frequency_Slot=2
    for request in requests:
        source = request['source']
        destination = request['destination']
        demand = request['demand']  
        
        
        try:   
            primary_path = nx.dijkstra_path(graph, source=source, target=destination, weight='weight')
            total_distance_primary = sum(graph[primary_path[i]][primary_path[i+1]]['weight'] for i in range(len(primary_path) - 1))
            
            graph_backup = graph.copy()
            for i in range(len(primary_path) - 1):
                graph_backup.remove_edge(primary_path[i], primary_path[i+1])
            try:
                backup_path = nx.dijkstra_path(graph_backup, source=source, target=destination, weight='weight')
                total_distance_backup = sum(graph_backup[backup_path[i]][backup_path[i+1]]['weight'] for i in range(len(backup_path) - 1))
            except nx.NetworkXNoPath:
                backup_path = None
                total_distance_backup = None
            
            regenerations_primary = 0
            regenerations_backup = 0
            current_distance_primary = 0
            current_distance_backup = 0
            regeneration_nodes_primary = []
            regeneration_nodes_backup = []
            path_segments_primary = []
            path_segments_backup = []
            segment_start_primary = source
            segment_start_backup = source
            
#             FOR PRIMARY PATH
            
            for i in range(1, len(primary_path)):
                link_distance = graph[primary_path[i-1]][primary_path[i]]['weight']
                current_distance_primary += link_distance 
                N = (demand//Frequency_Slot_Capacity)+Guard_Frequency_Slot
                Total_Frequency_Slot+=N
                
                
                if current_distance_primary > distance_threshold:
                    regenerations_primary += 1
                    total_regen_primary+=1
                    regeneration_nodes_primary.append(primary_path[i-1])
                    
                    if segment_start_primary != primary_path[i-1]:
                        path_segments_primary.append((segment_start_primary, primary_path[i-1]))
                    segment_start_primary = primary_path[i-1]
                        
                    current_distance_primary -= distance_threshold
            
            path_segments_primary.append((segment_start_primary, destination))
            
#             FOR BACKUP PATH

            if backup_path:
                for i in range(1, len(backup_path)):
                    link_distance = graph_backup[backup_path[i-1]][backup_path[i]]['weight']
                    current_distance_backup += link_distance
                    
                    if current_distance_backup > distance_threshold:
                        regenerations_backup += 1
                        regeneration_nodes_backup.append(backup_path[i-1])
                        
                        if segment_start_backup != backup_path[i-1]:
                            path_segments_backup.append((segment_start_backup, backup_path[i-1]))
                        segment_start_backup = backup_path[i-1]
                        
                        current_distance_backup -= distance_threshold
                
                path_segments_backup.append((segment_start_backup, destination))
            
            results.append({
                'source': source,
                'destination': destination,
                'demand':demand,
                
                'primary_path': primary_path,
                'total_distance_primary': total_distance_primary,
                'path_segments_primary': path_segments_primary,
                'regenerations_primary': regenerations_primary,
                'regeneration_nodes': regeneration_nodes_primary,
                
                'backup_path':backup_path,
                'total_distance_backup': total_distance_backup,
                'path_segments_backup': path_segments_backup,
                'regenerations_backup': regenerations_backup,
                'regeneration_nodes_backup': regeneration_nodes_backup,
                
                'Total_Regenerations_Primary' : total_regen_primary,
                'Total_Frequency_Slot' : Total_Frequency_Slot,
                'Spectrum_Allocated':int(N),
                
            })
        except nx.NetworkXNoPath:
            print(f"No path found between {source} and {destination}.")
            results.append({
                'source': source,
                'destination': destination,
                'demand':None,
                'primary_path': None,
                'total_distance_primary': None,
                'backup_path':None,
                'total_distance_backup': None,
                'path_segments_primary': None,
                'regenerations_primary': None,
                'regeneration_nodes': None,
                'Total_Regenerations_Primary' : None,
                'Total_Frequency_slot' : None,
                'Spectrum_Allocated':None,
            })
    return results

network_graph=graph

distance_threshold = 2000  
Frequency_Slot_Capacity = 25

results = sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity)

for result in results:
    print("\nRequest from {} to {}".format(result['source'], result['destination']))
    print("  Demand:",result["demand"])
    print("************************************************************")
    print("  Path (primary):", result['primary_path'])
    print("  Total Distance (primary):", result['total_distance_primary'])
    print("  Regenerations Required (primary):", result['regenerations_primary'])
    print("  Regeneration_at_nodes (primary):",result["regeneration_nodes"])
    print("  Spectrum Allocated:",result["Spectrum_Allocated"])
    print("  Segment (primary):",result["path_segments_primary"] )
    
    print("\n  Path (backup):", result['backup_path'])
    print("  Total Distance (Backup):", result['total_distance_backup'])
    print("  Regenerations Required (backup):", result['regenerations_backup'])
    print("  Regeneration_at_nodes (backup):", result["regeneration_nodes_backup"])
    print("  Segment (backup):", result["path_segments_backup"])
    
    
print("\n\nTotal_Regenerations:" ,result["Total_Regenerations_Primary"])
print("Total_Frequency_Slot:" ,result["Total_Frequency_Slot"])


Request from 6 to 2
  Demand: 62.5
************************************************************
  Path (primary): [6, 2]
  Total Distance (primary): 1000
  Regenerations Required (primary): 0
  Regeneration_at_nodes (primary): []
  Spectrum Allocated: 4
  Segment (primary): [(6, 2)]

  Path (backup): [6, 3, 2]
  Total Distance (Backup): 1100
  Regenerations Required (backup): 0
  Regeneration_at_nodes (backup): []
  Segment (backup): [(6, 2)]

Request from 10 to 20
  Demand: 50.0
************************************************************
  Path (primary): [10, 11, 15, 20]
  Total Distance (primary): 2900
  Regenerations Required (primary): 1
  Regeneration_at_nodes (primary): [15]
  Spectrum Allocated: 4
  Segment (primary): [(10, 15), (15, 20)]

  Path (backup): [10, 14, 15, 21, 20]
  Total Distance (Backup): 3000
  Regenerations Required (backup): 1
  Regeneration_at_nodes (backup): [15]
  Segment (backup): [(10, 15), (15, 20)]

Request from 22 to 3
  Demand: 12.5
****************

# SPECTRUM GENERATION

In [3]:
import numpy as np

total_segments=0
segment_names = []

for result in results:
    if result["path_segments_primary"]:
        for segment in result["path_segments_primary"]:
            total_segments += 1
            segment_names.append(f"segment_primary_{segment[0]}_{segment[1]}")

    # if result["path_segments_backup"]:
    #     for segment in result["path_segments_backup"]:
    #         total_segments += 1
    #         segment_names.append(f"segment_backup_{segment[0]}_{segment[1]}")

array = np.zeros((total_segments, 100))

print("\nTotal Segments:", total_segments)
print("Segment Names:")
for i, name in enumerate(segment_names):
    print(f"Row {i}: {name}")

print("\n2D Array initialized to zeros:")
print(array)
print(array.shape)



Total Segments: 87
Segment Names:
Row 0: segment_primary_6_2
Row 1: segment_primary_10_15
Row 2: segment_primary_15_20
Row 3: segment_primary_22_15
Row 4: segment_primary_15_8
Row 5: segment_primary_8_3
Row 6: segment_primary_2_3
Row 7: segment_primary_12_15
Row 8: segment_primary_4_7
Row 9: segment_primary_7_13
Row 10: segment_primary_13_23
Row 11: segment_primary_4_6
Row 12: segment_primary_14_16
Row 13: segment_primary_23_21
Row 14: segment_primary_21_14
Row 15: segment_primary_14_10
Row 16: segment_primary_17_12
Row 17: segment_primary_20_11
Row 18: segment_primary_11_8
Row 19: segment_primary_8_0
Row 20: segment_primary_7_6
Row 21: segment_primary_6_5
Row 22: segment_primary_11_8
Row 23: segment_primary_8_5
Row 24: segment_primary_8_10
Row 25: segment_primary_6_0
Row 26: segment_primary_14_11
Row 27: segment_primary_11_9
Row 28: segment_primary_3_8
Row 29: segment_primary_8_15
Row 30: segment_primary_15_14
Row 31: segment_primary_11_13
Row 32: segment_primary_9_13
Row 33: segment

# CREATING LINK DISJOINT GROUP OF PRIMARY AND BACKUP PATH

In [4]:
disjoint_groups = []
backup_disjoint_groups = [] 

def are_paths_link_disjoint(path1, path2, graph):
    for i in range(len(path1) - 1):
        edge1 = (path1[i], path1[i + 1])
        edge1_reverse = (path1[i + 1], path1[i])
        for j in range(len(path2) - 1):
            edge2 = (path2[j], path2[j + 1])
            edge2_reverse = (path2[j + 1], path2[j])
            if edge1 == edge2 or edge1 == edge2_reverse:
                return False
    return True

def is_path_already_in_groups(path, groups):
    for group in groups:
        for existing_path in group:
            if path == existing_path:
                return True
    return False

primary_to_backup = {}

for result in results:
    primary_path = result['primary_path']
    backup_path = result.get('backup_path')
    if primary_path is not None:
        primary_to_backup[tuple(primary_path)] = backup_path
        if is_path_already_in_groups(primary_path, disjoint_groups):
            continue
        
        placed = False
        for group in disjoint_groups:
            is_disjoint = True
            for existing_path in group:
                if not are_paths_link_disjoint(primary_path, existing_path, network_graph):
                    is_disjoint = False
                    break
            if is_disjoint:
                group.append(primary_path)
                placed = True
                break
        if not placed:
            disjoint_groups.append([primary_path])

for primary_group in disjoint_groups:
    backup_group = []
    for primary_path in primary_group:
        backup_path = primary_to_backup.get(tuple(primary_path))
        if backup_path:
            backup_group.append(backup_path)
    backup_disjoint_groups.append(backup_group)

print("\nDisjoint Groups of Primary Paths:")
for idx, group in enumerate(disjoint_groups, 1):
    print(f"\nGroup {idx}:\n")
    for path in group:
        print(path)

print("\nDisjoint Groups of Backup Paths:")
for idx, group in enumerate(backup_disjoint_groups, 1):
    print(f"\nBackup Group {idx}:\n")
    for path in group:
        print(path)



Disjoint Groups of Primary Paths:

Group 1:

[6, 2]
[10, 11, 15, 20]
[2, 3]
[4, 7, 9, 13, 17, 23]
[4, 3, 6]
[14, 15, 16]
[7, 6, 5]
[11, 8, 5]
[8, 10]
[11, 12, 13]
[22, 21, 20, 19]
[1, 5, 10, 18]
[5, 0]
[9, 8, 6]

Group 2:

[22, 21, 15, 11, 8, 6, 3]
[17, 13, 12]
[6, 5, 0]
[9, 13]
[6, 7]
[11, 10, 18]
[21, 16, 12, 9, 7, 4]
[11, 12]
[20, 15, 14]
[10, 5]
[0, 1, 2, 3]

Group 3:

[12, 11, 15]
[23, 22, 21, 15, 14, 10]
[17, 13, 9, 7]
[6, 8]
[1, 2, 3]
[12, 16, 21, 20, 19]

Group 4:

[20, 15, 11, 8, 5, 0]
[14, 15, 21]
[13, 9, 7]

Group 5:

[14, 15, 11, 12, 9]
[2, 4, 7, 9]
[0, 5, 8, 9, 13]

Group 6:

[3, 6, 8, 11, 15, 14]

Group 7:

[21, 15, 11, 8]
[15, 14, 10]

Group 8:

[14, 15, 11, 8, 6, 3]

Group 9:

[12, 11, 8, 6, 3]
[14, 15, 21, 22]

Group 10:

[16, 15, 11, 8, 5, 1]

Group 11:

[21, 15, 11, 8, 5]

Group 12:

[2, 6, 8, 11, 15, 20, 19]

Group 13:

[16, 15, 11, 8]

Disjoint Groups of Backup Paths:

Backup Group 1:

[6, 3, 2]
[10, 14, 15, 21, 20]
[2, 4, 3]
[4, 3, 6, 8, 11, 15, 21, 22, 23]
[4, 2

# SPECTRUM ALLOCATION

In [5]:
counter = 0
for result in results:
    if result["path_segments_primary"]:
        for segment in result["path_segments_primary"]:
            print (f"{segment} -> {result["Spectrum_Allocated"]}")
            counter += 1

print(counter)


(6, 2) -> 4
(10, 15) -> 4
(15, 20) -> 4
(22, 15) -> 2
(15, 8) -> 2
(8, 3) -> 2
(2, 3) -> 3
(12, 15) -> 2
(4, 7) -> 2
(7, 13) -> 2
(13, 23) -> 2
(4, 6) -> 3
(14, 16) -> 5
(23, 21) -> 3
(21, 14) -> 3
(14, 10) -> 3
(17, 12) -> 3
(20, 11) -> 4
(11, 8) -> 4
(8, 0) -> 4
(7, 6) -> 3
(6, 5) -> 3
(11, 8) -> 3
(8, 5) -> 3
(8, 10) -> 3
(6, 0) -> 3
(14, 11) -> 2
(11, 9) -> 2
(3, 8) -> 3
(8, 15) -> 3
(15, 14) -> 3
(11, 13) -> 3
(9, 13) -> 3
(6, 7) -> 5
(21, 11) -> 4
(11, 8) -> 4
(14, 11) -> 4
(11, 6) -> 4
(6, 3) -> 4
(22, 19) -> 3
(11, 10) -> 3
(10, 18) -> 3
(12, 8) -> 3
(8, 3) -> 3
(14, 21) -> 3
(17, 13) -> 3
(13, 7) -> 3
(16, 11) -> 5
(11, 8) -> 5
(8, 1) -> 5
(1, 5) -> 2
(5, 10) -> 2
(10, 18) -> 2
(9, 13) -> 2
(21, 12) -> 2
(12, 7) -> 2
(7, 4) -> 2
(11, 12) -> 2
(21, 11) -> 3
(11, 5) -> 3
(20, 14) -> 3
(5, 0) -> 5
(9, 6) -> 3
(15, 10) -> 5
(6, 8) -> 3
(11, 8) -> 4
(8, 5) -> 4
(2, 8) -> 4
(8, 15) -> 4
(15, 19) -> 4
(13, 7) -> 4
(10, 5) -> 4
(2, 7) -> 2
(7, 9) -> 2
(0, 2) -> 3
(2, 3) -> 3
(1, 3) ->

In [9]:
array = np.zeros((counter, 20))
index = 0
for result in results:
    if result["path_segments_primary"]:
        for segment in result["path_segments_primary"]:
            array[index, :result["Spectrum_Allocated"]] = 1
            print(f"{segment} -> {array[index]}")
            index += 1

(22, 23) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(10, 14) -> [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(13, 11) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(16, 13) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(12, 8) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(8, 5) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(5, 0) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(17, 13) -> [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(13, 7) -> [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(7, 2) -> [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(4, 6) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(6, 5) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(19, 18) -> [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(22, 15) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [16]:
import numpy as np

# Initialize the array
array = np.zeros((counter, 20))
index = 0

track=0

# Dictionary to keep track of segment indices and their last allocated position
segment_indices = {}
segment_last_allocated = {}

for result in results:
    if result["path_segments_primary"]:
        for segment in result["path_segments_primary"]:
            if segment in segment_indices:
                # Update the existing allocation
                existing_index = segment_indices[segment]
                start_position = segment_last_allocated[segment]
                end_position = start_position + result["Spectrum_Allocated"]

                track+=result["Spectrum_Allocated"]

                array[existing_index, :track] = 0

                array[existing_index, start_position:end_position] = 1
                segment_last_allocated[segment] = end_position
            else:
                # Allocate spectrum for the new segment
                array[index, :result["Spectrum_Allocated"]] = 1
                segment_indices[segment] = index
                segment_last_allocated[segment] = result["Spectrum_Allocated"]
                index += 1
            print(f"{segment} -> {array[segment_indices[segment]]}")

(22, 23) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(10, 14) -> [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(13, 11) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(16, 13) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(12, 8) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(8, 5) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(5, 0) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(17, 13) -> [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(13, 7) -> [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(7, 2) -> [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(4, 6) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(6, 5) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(19, 18) -> [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(22, 15) -> [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0